# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
from nltk.metrics.distance import edit_distance

In [3]:
s1 = "ПИ19-2"
s2 = "ПИ19-1"
edit_distance(s1, s2)

1

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [6]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''
with open ("./litw-win.txt", "r", encoding='windows-1251') as fp:
    words = [line.strip().split()[-1] for line in fp]
words[-5:]

['высокопревосходительства',
 'попреблагорассмотрительст',
 'попреблагорассмотрительствующемуся',
 'убегающих',
 'уменьшившейся']

In [7]:
' '.join([min(words, key=lambda w: edit_distance(w, t)) for t in text.split()])

'с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий'

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [8]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [9]:
stemmer = SnowballStemmer('russian')

for word in word_tokenize(text):
    result = stemmer.stem(word)
    print(result)

с
велечайш
усил
выбра
из
поток
убега
люд
кутуз
со
свит
уменьшевш
вдво
поеха
на
звук
выстрел
русск
оруд


In [10]:
morph = pymorphy2.MorphAnalyzer()
for word in word_tokenize(text):
    result = morph.parse(word)[0].normalized.word
    print(result)

с
велечайший
усилие
выбраться
из
поток
убегать
человек
кутузов
с
свита
уменьшевшийся
вдвое
поехать
на
звук
выстрел
русский
орудие


3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [11]:
text = '''Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. '''
sents = sent_tokenize(text)
sents

['Считайте слова из файла `litw-win.txt` и запишите их в список `words`.',
 'В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`.',
 'Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`.']

In [12]:
cv = CountVectorizer()
cv.fit(sents)
sents_cv = cv.transform(sents).toarray()
sents_cv

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]], dtype=int64)

In [13]:
cv.vocabulary_

{'считайте': 32,
 'слова': 24,
 'из': 12,
 'файла': 33,
 'litw': 0,
 'win': 2,
 'txt': 1,
 'запишите': 11,
 'их': 14,
 'список': 31,
 'words': 3,
 'заданном': 9,
 'предложении': 22,
 'исправьте': 13,
 'все': 5,
 'опечатки': 21,
 'заменив': 10,
 'опечатками': 20,
 'на': 16,
 'ближайшие': 4,
 'смысле': 27,
 'расстояния': 23,
 'левенштейна': 15,
 'ним': 18,
 'списка': 29,
 'что': 34,
 'слове': 25,
 'есть': 8,
 'опечатка': 19,
 'если': 7,
 'данное': 6,
 'слово': 26,
 'не': 17,
 'содержится': 28,
 'списке': 30}

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [16]:
import pandas as pd

preprocessed_descriptions = pd.read_csv("./preprocessed_descriptions.csv")
preprocessed_descriptions.head()

,id,name,preprocessed_descriptions
0,0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,2,i can t believe it s spinach,these were so go it surprised even me
3,3,italian gut busters,my sisterinlaw made these for us at a family g...
4,4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...


In [27]:
from functools import reduce
words = reduce(lambda x,y: x + y, [word_tokenize(item) for item in preprocessed_descriptions["preprocessed_descriptions"].to_list() if isinstance(item, str)])
words[:20]

['an',
 'original',
 'recipe',
 'created',
 'by',
 'chef',
 'scott',
 'meskan',
 'georges',
 'at',
 'the',
 'cove',
 'we',
 'enjoyed',
 'this',
 'when',
 'we',
 'visited',
 'this',
 'restaurant']

In [29]:
words = list(set(words))
words[:20]

['bueno',
 'recipesliced',
 'exhausting',
 'cookedmashed',
 'differenttry',
 'homemaker',
 'smokies',
 'doodlefathr',
 'cored',
 'lesssweet',
 'gutsy',
 'cous',
 'splash',
 'spezie',
 'fruits',
 'force',
 'beacha',
 'encore',
 'mangoflavored',
 'recfoodbaking']

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [40]:
import random as rd

pairs = [rd.choices(words, k=2) for _ in range(5)]
pairs

[['sid', 'et'],
 ['term', 'spanakopita'],
 ['sackthis', 'thornberryit'],
 ['saladises', 'bloody'],
 ['wintertime', 'burritostyle']]

In [41]:
h_s = [edit_distance(*pair) for pair in pairs]
h_s

[3, 11, 11, 8, 9]

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [42]:
def k_nearest(word, k=1):
    w_new = sorted(words, key=lambda w: edit_distance(w, word))
    return w_new[:k]

k_nearest('term', k=3)

['term', 'teri', 'team']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [48]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ivanb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [62]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

stemmed_words = [stemmer.stem(word) for word in words]
normalized_word = [lemmatizer.lemmatize(word) for word in words]
df = pd.DataFrame(list(zip(words, stemmed_words, normalized_word))[10:20], columns=['word', 'stemmed_word', 'normalized_word'])
df = df.set_index('stemmed_word')
df.head()

,word,normalized_word
stemmed_word,,
gutsy,gutsy,gutsy
cous,cous,cous
splash,splash,splash
spezie,spezie,spezie
fruits,fruits,fruit


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [64]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivanb\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [71]:
from nltk.corpus import stopwords

st_w = stopwords.words()
texts = reduce(lambda x, y: x + ' ' + y, [item for item in preprocessed_descriptions["preprocessed_descriptions"].to_list() if isinstance(item, str)])
tokens = word_tokenize(texts)
l = len(tokens)
l_new = 0
words_dict = {}
words_dict_stop = {}
for w in tokens:
    if w not in st_w:
        words_dict_stop[w] = words_dict_stop.get(w, 0) + 1
        l_new += 1
    words_dict[w] = words_dict.get(w, 0) + 1

In [72]:
print(f'Доля стоп слов - {(l - l_new) / l}')
print(f'Топ 10 слов со стоп словами: {" ".join(sorted(words_dict.keys(), key=lambda x: words_dict[x], reverse=True)[:10])}')
print(f'Топ 10 слов без стоп слов: {" ".join(sorted(words_dict_stop.keys(), key=lambda x: words_dict_stop[x], reverse=True)[:10])}')

Доля стоп слов - 0.47034287456488355
Топ 10 слов со стоп словами: the a and this i to is it of for
Топ 10 слов без стоп слов: recipe make time use great like easy made good dish


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

rd_recepies = rd.choices(preprocessed_descriptions["preprocessed_descriptions"], k=5)
rd_recepies

['this recipe originally came from the usa weekend paper i changed a few things around to suit us and this is the end result',
 'a well known and delicious arabic sweet commonly eaten in countries like palestine and jordan this is a variation that my family does that does not contain sweet cheese unlike other kenafe recipes this one taste delicious reheated',
 'these are delicious cornish game hens they take awhile to prepare but are worth it i have tripled the recipe these are a family favorite i found the recipe some time ago on the web',
 'this is a fun recipe  kids will love it  i serve with a jarred marinara sauce that i jazz up a little  i have also tried this recipe using pork chops cut into thin strips  both ways are excellent  enjoy',
 'the pizza stone we bought came with recipes  this is one of them  i havent tried it yet but it looks really really good']

In [84]:
vectorizer = TfidfVectorizer()
vectorizer.fit(rd_recepies)
sent_vec = vectorizer.transform(rd_recepies)
sent_vec = sent_vec.toarray()

3.2 Вычислите близость между каждой парой рецептов, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [86]:
from scipy.spatial import distance
d_dist = {}
for i in range(sent_vec.shape[0] - 1):
    for j in range(i + 1, sent_vec.shape[0]):
        d_dist[(rd_recepies[i], rd_recepies[j])] = distance.cosine(sent_vec[i], sent_vec[j])
print(d_dist)

{('this recipe originally came from the usa weekend paper i changed a few things around to suit us and this is the end result', 'a well known and delicious arabic sweet commonly eaten in countries like palestine and jordan this is a variation that my family does that does not contain sweet cheese unlike other kenafe recipes this one taste delicious reheated'): 0.8937396582041199, ('this recipe originally came from the usa weekend paper i changed a few things around to suit us and this is the end result', 'these are delicious cornish game hens they take awhile to prepare but are worth it i have tripled the recipe these are a family favorite i found the recipe some time ago on the web'): 0.8355859919279524, ('this recipe originally came from the usa weekend paper i changed a few things around to suit us and this is the end result', 'this is a fun recipe  kids will love it  i serve with a jarred marinara sauce that i jazz up a little  i have also tried this recipe using pork chops cut int

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат

In [88]:
m = max(d_dist.keys(), key=lambda x: d_dist[x])
print(m)
print(d_dist[m])

('a well known and delicious arabic sweet commonly eaten in countries like palestine and jordan this is a variation that my family does that does not contain sweet cheese unlike other kenafe recipes this one taste delicious reheated', 'these are delicious cornish game hens they take awhile to prepare but are worth it i have tripled the recipe these are a family favorite i found the recipe some time ago on the web')
0.9476740474958096
